In [12]:
# Allow import of modules from src/
import sys
sys.path.append("..")

In [13]:
from pathlib import Path

import logging

import yappi

from pandas import DataFrame

import copy

from muCh import muCh
from checking import Checker
from parsing import Parser

In [14]:
systems_dir = Path("../../material/dining/")
formulas_dir = Path("../../material/dining/")

results_path = Path("../../profiling/demanding/2022-12-16_15-07.csv")

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

In [15]:
algorithm = Checker.Algorithm.NAIVE

system_paths = [file for file in systems_dir.iterdir() if file.suffix == '.aut']
formula_paths = [file for file in formulas_dir.iterdir() if file.suffix == '.mcf']

In [16]:
formulas = {}
for formula_path in formula_paths:
    logging.info("Parsing formula '%s'." % formula_path.name)
    formulas[formula_path.name] = Parser.parse_formula(formula_path)

systems = {}
for system_path in system_paths:
    logging.info("Parsing system '%s'." % system_path.name)
    systems[system_path.name] = Parser.parse_system(system_path)

15:08:11:INFO:Parsing formula 'invariantly_inevitably_eat.mcf'.
15:08:12:INFO:Parsing formula 'invariantly_plato_starves.mcf'.
15:08:12:INFO:Parsing formula 'invariantly_possibly_eat.mcf'.
15:08:13:INFO:Parsing formula 'plato_infinitely_often_can_eat.mcf'.
15:08:13:INFO:Parsing system 'dining_2.aut'.


{'invariantly_inevitably_eat.mcf': {'nu': [{'var': 'X'}, {'and': [{'and': [{'box': ['i', {'var': 'X'}]}, {'and': [{'box': ['plato', {'var': 'X'}]}, {'box': ['others', {'var': 'X'}]}]}]}, {'mu': [{'var': 'Y'}, {'and': [{'box': ['i', {'var': 'Y'}]}, {'or': [{'diamond': ['plato', {'val': True}]}, {'diamond': ['others', {'val': True}]}]}]}]}]}]}, 'invariantly_plato_starves.mcf': {'nu': [{'var': 'X'}, {'and': [{'and': [{'box': ['i', {'var': 'X'}]}, {'box': ['others', {'var': 'X'}]}]}, {'nu': [{'var': 'Y'}, {'or': [{'diamond': ['i', {'var': 'Y'}]}, {'diamond': ['others', {'var': 'Y'}]}]}]}]}]}, 'invariantly_possibly_eat.mcf': {'nu': [{'var': 'X'}, {'and': [{'and': [{'box': ['i', {'var': 'X'}]}, {'and': [{'box': ['plato', {'var': 'X'}]}, {'box': ['others', {'var': 'X'}]}]}]}, {'mu': [{'var': 'Y'}, {'or': [{'or': [{'diamond': ['i', {'var': 'Y'}]}, {'diamond': ['others', {'var': 'Y'}]}]}, {'diamond': ['plato', {'val': True}]}]}]}]}]}, 'plato_infinitely_often_can_eat.mcf': {'nu': [{'var': 'X'}, 

15:08:13:INFO:Parsing system 'dining_3.aut'.
15:08:13:INFO:Parsing system 'dining_4.aut'.
15:08:14:INFO:Parsing system 'dining_5.aut'.
15:08:15:INFO:Parsing system 'dining_6.aut'.
15:08:20:INFO:Parsing system 'dining_7.aut'.


In [ ]:
statistics = []

for system_name, system in sorted(systems.items(), key=lambda x: int(x[0].split("_")[1].split(".")[0])):
    for formula_name, formula in formulas.items():
        for algorithm in Checker.Algorithm:
            logging.info("Checking system %s on formula %s with algorithm %s." % (system_name, formula_name, algorithm.name))
            
            # Do not let solver modify our formulas.
            system_copy = copy.deepcopy(system)
            formula_copy = copy.deepcopy(formula)

            yappi.clear_stats()
            yappi.start()
            satisfied, _, ifp = Checker.solver(system_copy, formula_copy, algorithm)
            yappi.stop()
            
            ttot = yappi.get_func_stats(filter_callback=lambda x: yappi.func_matches(x, [Checker.solver]))[0].ttot

            statistics.append((system_name, formula_name, algorithm.name, satisfied, ifp, ttot))

15:06:20:INFO:Checking system dining_2.aut on formula invariantly_inevitably_eat.mcf with algorithm NAIVE.
15:06:20:INFO:solver. ifp: 9
15:06:20:INFO:collect. ifp: 9
15:06:20:INFO:Checking system dining_2.aut on formula invariantly_inevitably_eat.mcf with algorithm EMERSON_LEI.
15:06:20:INFO:solver. ifp: 7
15:06:20:INFO:collect. ifp: 7
15:06:20:INFO:Checking system dining_2.aut on formula invariantly_plato_starves.mcf with algorithm NAIVE.
15:06:20:INFO:solver. ifp: 24
15:06:20:INFO:collect. ifp: 24
15:06:20:INFO:Checking system dining_2.aut on formula invariantly_plato_starves.mcf with algorithm EMERSON_LEI.
15:06:20:INFO:solver. ifp: 14
15:06:20:INFO:collect. ifp: 14
15:06:20:INFO:Checking system dining_2.aut on formula invariantly_possibly_eat.mcf with algorithm NAIVE.
15:06:20:INFO:solver. ifp: 63
15:06:20:INFO:collect. ifp: 63
15:06:20:INFO:Checking system dining_2.aut on formula invariantly_possibly_eat.mcf with algorithm EMERSON_LEI.
15:06:20:INFO:solver. ifp: 21
15:06:20:INFO:c

KeyboardInterrupt: 

In [ ]:
df = DataFrame(statistics, columns = ["System", "Formula", "Algorithm", "satisfied", "ifp", "ttot"])

In [ ]:
# Print profiling results
df

,System,Formula,Algorithm,satisfied,ifp,ttot
0,demanding_children_2.aut,attention_to_other_children.mcf,NAIVE,True,4,0.015625
1,demanding_children_2.aut,attention_to_other_children.mcf,EMERSON_LEI,True,4,0.015625
2,demanding_children_2.aut,eventual_questions.mcf,NAIVE,True,5,0.015625
3,demanding_children_2.aut,eventual_questions.mcf,EMERSON_LEI,True,5,0.015625
4,demanding_children_2.aut,no_skipping_wisdom.mcf,NAIVE,True,3,0.015625
5,demanding_children_2.aut,no_skipping_wisdom.mcf,EMERSON_LEI,True,2,0.015625
6,demanding_children_3.aut,attention_to_other_children.mcf,NAIVE,True,4,0.015625
7,demanding_children_3.aut,attention_to_other_children.mcf,EMERSON_LEI,True,4,0.015625
8,demanding_children_3.aut,eventual_questions.mcf,NAIVE,True,6,0.015625
9,demanding_children_3.aut,eventual_questions.mcf,EMERSON_LEI,True,6,0.015625


In [ ]:
# Save data frame
df.to_csv(results_path)